In [1]:
## importing libraries
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [2]:
## reading the files and loading them into dataframes.
train = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Train.csv')
test= pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Test.csv')
sample = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/sample_submission.csv')
mask = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/unlinked_masked_final.csv')
variabs = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/VariableDefinitions.csv')

FileNotFoundError: [Errno 2] File b'C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Train.csv' does not exist: b'C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Train.csv'

In [ ]:
## Transform dates types from 'object' to 'datetime'
train.TransactionStartTime=pd.to_datetime(train.TransactionStartTime)
test.TransactionStartTime=pd.to_datetime(test.TransactionStartTime)
train.IssuedDateLoan=pd.to_datetime(train.IssuedDateLoan)
test.IssuedDateLoan=pd.to_datetime(test.IssuedDateLoan)
train.PaidOnDate=pd.to_datetime(train.PaidOnDate)
train.DueDate=pd.to_datetime(train.DueDate)

In [ ]:
train.corr()

In [ ]:
## creating variables to transfer the information contained in the rows of the same transaction.
train['Number_Of_Split_Payments'] = 0 ## this is a count on the number of payments on the same loan. It will take a 0 for singled-rowed transactions, 1+ for multi-row transacs.
#train['Sum_Diff_Time_Payments'] = 0 ## I'm thinking of summing the delays between all payments made on a loan. It will take 0 for loans paid in a single time, 1+ for multiple payments on the same loan.
test['Number_Of_Split_Payments']=0
#test['Sum_Diff_Time_Payments']=0

In [ ]:
## creating the feature : number of split payments on a loan.
train['Number_Of_Split_Payments']=train['TransactionId'].map(train.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])
test['Number_Of_Split_Payments']=test['TransactionId'].map(test.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])

In [ ]:
train.drop(train[(train.TransactionId=='TransactionId_703')|((train.TransactionId=='TransactionId_927'))].index,axis=0,inplace=True)

In [ ]:
## Lets drop the duplicate rows with the same transaction ID and keep the last one. (as in with the latest payment installment )
train.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)
test.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)

In [ ]:
train.drop(['CountryCode','Currency','CurrencyCode','SubscriptionId','ProviderId','ChannelId'],axis=1,inplace=True)
test.drop(['CountryCode','CurrencyCode','SubscriptionId','ProviderId','ChannelId'],axis=1,inplace=True)

### Feature engineering

In [ ]:
train['Count_Rejected_Loans'] = train['CustomerId'].map(train[train.TransactionStatus==0].groupby('CustomerId').LoanId.size())
test['Count_Rejected_Loans'] = test['CustomerId'].map(train[train.TransactionStatus==0].groupby('CustomerId').LoanId.size())
## then we should impute the columns of customers that were not found in the rejected list with 0 as in they have never been rejected.
train.Count_Rejected_Loans.fillna(value=0,inplace=True)
test.Count_Rejected_Loans.fillna(value=0,inplace=True)

In [ ]:
## group train/test together to perform cumulative count
all_data=pd.concat((train,test)).copy()
## Initialize and compute values for the new feature
all_data['Cumulative_Reject']=0
all_data.loc[all_data.TransactionStatus==0,'Cumulative_Reject'] = all_data[all_data.TransactionStatus==0].groupby('CustomerId').cumcount()
## Separate all_data into train and test
train1=all_data[:len(train)]
test1=all_data[len(train):]
train['Cumulative_Reject']=0
test['Cumulative_Reject']=0
train['Cumulative_Reject']=train1['Cumulative_Reject']
test['Cumulative_Reject']=test1['Cumulative_Reject']

In [ ]:
purchasestats=train[train.TransactionStatus==0].groupby('CustomerId').Value.agg(('mean','std','min','max'))
train['prchs_mean']=train['CustomerId'].map(purchasestats['mean'])
train['prchs_std']=train['CustomerId'].map(purchasestats['std'])
train['prchs_max']=train['CustomerId'].map(purchasestats['max'])
train['prchs_min']=train['CustomerId'].map(purchasestats['min'])
test['prchs_mean']=test['CustomerId'].map(purchasestats['mean'])
test['prchs_std']=test['CustomerId'].map(purchasestats['std'])
test['prchs_max']=test['CustomerId'].map(purchasestats['max'])
test['prchs_min']=test['CustomerId'].map(purchasestats['min'])

In [ ]:
valuegroups=mask.groupby('CustomerId').Value.agg(('mean','std','min','max','count'))
train['mean_cus_transac']=train['CustomerId'].map(valuegroups['mean'])
train['std_cus_transac']=train['CustomerId'].map(valuegroups['std'])
train['min_cus_transac']=train['CustomerId'].map(valuegroups['min'])
train['max_cus_transac']=train['CustomerId'].map(valuegroups['max'])
train['count_cus_transac']=train['CustomerId'].map(valuegroups['count'])
test['mean_cus_transac']=test['CustomerId'].map(valuegroups['mean'])
test['std_cus_transac']=test['CustomerId'].map(valuegroups['std'])
test['min_cus_transac']=test['CustomerId'].map(valuegroups['min'])
test['max_cus_transac']=test['CustomerId'].map(valuegroups['max'])
test['count_cus_transac']=test['CustomerId'].map(valuegroups['count'])

In [ ]:
train['Day_Of_Week']= train.TransactionStartTime.dt.weekday
test['Day_Of_Week'] =test.TransactionStartTime.dt.weekday
train['Day_in_month']=train.TransactionStartTime.dt.day
test['Day_in_month']=test.TransactionStartTime.dt.day

In [ ]:
from datetime import date
datemin = date(2018,9,21)
datemax= date(2019,7,17)
(datemax-datemin).days
datesinc=pd.DataFrame(columns=['date','inc_value'])
datesinc.loc[0,'inc_value']=1
datesinc.loc[0,'date']=datemin
from datetime import timedelta
for i in range(2,301):
    datesinc.loc[i-1,'inc_value']=i
    datesinc.loc[i-1,'date']=datemin + timedelta(days=i-1)
train['inc_value_date']=train.TransactionStartTime.dt.date.map(datesinc.set_index('date').inc_value)
test['inc_value_date']=test.TransactionStartTime.dt.date.map(datesinc.set_index('date').inc_value)

In [ ]:
train.inc_value_date = train.inc_value_date.astype(np.int64)
test.inc_value_date = test.inc_value_date.astype(np.int64)

In [ ]:
aa=train[(train.TransactionStatus==1)&(train.TransactionStartTime<train.DueDate)].groupby('CustomerId').agg(('count','mean','std','min','max')).Value
#train['number_transac_before_due']=train['CustomerId'].map(aa['count'])
train['before_due_mean'] = train['CustomerId'].map(aa['mean'])
train['before_due_std'] = train['CustomerId'].map(aa['std'])
train['before_due_min'] = train['CustomerId'].map(aa['min'])
train['before_due_max'] = train['CustomerId'].map(aa['max'])
test['before_due_mean'] = test['CustomerId'].map(aa['mean'])
test['before_due_std'] = test['CustomerId'].map(aa['std'])
test['before_due_min'] = test['CustomerId'].map(aa['min'])
test['before_due_max'] = test['CustomerId'].map(aa['max'])

In [ ]:
train['Cnt_missed_payment']=0
train.loc[train.DueDate<train.PaidOnDate,'Cnt_missed_payment']=train[train.DueDate<train.PaidOnDate].groupby('CustomerId').cumcount()
test['Cnt_missed_payment']=test['CustomerId'].map(train.groupby('CustomerId').agg('max').Cnt_missed_payment)

In [ ]:
#train['IssuedDateLoan']=pd.to_datetime(train.IssuedDateLoan)
#train['Time_till_first_transac_onloan'] = (train['TransactionStartTime']-train['IssuedDateLoan']).astype('timedelta64[h]')
#train[(train['TransactionStartTime']-train['IssuedDateLoan']).astype('timedelta64[h]')<0]
#train['Factor_Loan']= train

In [ ]:
train=train[train.IsDefaulted.notnull()]

In [ ]:
mask.columns

In [ ]:
test.columns

In [3]:
train[train.IsDefaulted==1]

NameError: name 'train' is not defined

In [25]:
train.loc[:,'new_customer']=0
test.loc[:,'new_customer']=0
train.loc[train.mean_cus_transac.isnull(),'new_customer']=1
test.loc[test.mean_cus_transac.isnull(),'new_customer']=1

In [60]:
features = [#'CustomerId', #'TransactionStartTime', 
            'Value', #'Amount',
       #'TransactionId', 
            #'BatchId', 
             #'ProductId',
    'mean_cus_transac','std_cus_transac', 'min_cus_transac', 'max_cus_transac', 
       #'ProductCategory', #'TransactionStatus', 
            #'IssuedDateLoan',
       #'LoanId', , 'LoanApplicationId', 'ThirdPartyId',
       #'Number_Of_Split_Payments', 
        #    'Count_Rejected_Loans', 
           #'Cumulative_Reject',
       #'prchs_mean', 'prchs_std',
    'prchs_max', 'prchs_min', 
         #   'InvestorId',
         #   'Day_Of_Week',
    'Day_in_month',#,'new_customer' 
    #'inc_value_date'
    #'before_due_mean'#, 'before_due_std',
       'before_due_min', 'before_due_max', 
            'count_cus_transac'#,'Cnt_missed_payment'
]

In [61]:
#oce = ce.OneHotEncoder(cols=['ProductId'])#,'ProductCategory','InvestorId'])
#tce = ce.TargetEncoder(cols=['CustomerId'],smoothing=40,min_samples_leaf=5)

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X = train[features]
X_test = test[features]
y=train.IsDefaulted.copy()

In [64]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,stratify=y,random_state=10)

In [65]:
#X_train = oce.fit_transform(X_train)
#X_train = tce.fit_transform(X_train,y_train)
#X_val = oce.transform(X_val)
#X_val = tce.transform(X_val)

In [73]:
xgb = XGBClassifier(max_depth=3,colsample_bytree=0.6,min_child_weight=10,learning_rate=0.25,n_estimators=100,objective = "binary:logistic")
xgb.fit(X_train,y_train)
val_pred=xgb.predict_proba(X_val)[:,1]
print(roc_auc_score(y_val,val_pred))

0.9894667544437129


In [74]:
#X = oce.fit_transform(X)
#X = tce.fit_transform(X_train,y_train)
#X_test = test[features]
#X_test = oce.transform(X_test)
#X_test = tce.transform(X_test)

In [75]:
xgb.fit(X,y)
test_pred=xgb.predict_proba(X_test)[:,1]

In [76]:
sample_submission = pd.DataFrame(columns=['TransactionId','IsDefaulted'])
sample_submission['TransactionId'] = test['TransactionId']
sample_submission['IsDefaulted'] = test_pred

In [77]:
sample_submission.to_csv('C:/Users/Amine/Desktop/STB_XGB_Thleth8.csv',index=False)

In [68]:
train.describe()

,Value,Amount,TransactionStatus,AmountLoan,IsFinalPayBack,IsThirdPartyConfirmed,IsDefaulted,Number_Of_Split_Payments,Count_Rejected_Loans,Cumulative_Reject,...,max_cus_transac,Day_Of_Week,Day_in_month,inc_value_date,before_due_mean,before_due_std,before_due_min,before_due_max,Cnt_missed_payment,new_customer
count,1.153000e+03,1.153000e+03,1153.0,1.153000e+03,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.0,...,1073.000000,1153.000000,1153.000000,1153.000000,1.153000e+03,1.043000e+03,1.153000e+03,1.153000e+03,1153.000000,1153.000000
mean,2.128621e+04,-2.115861e+04,1.0,1.043774e+04,0.934952,0.980052,0.061578,1.290546,1.794449,0.0,...,52570.672880,2.680833,16.111015,124.830876,2.127673e+04,7.890890e+03,1.552366e+04,3.183914e+04,0.464874,0.069384
std,9.849843e+04,9.848352e+04,0.0,8.419850e+04,0.246717,0.139882,0.240493,1.062977,3.567100,0.0,...,74088.996842,1.846202,8.884179,39.034529,8.305972e+04,7.806406e+04,6.535204e+04,1.228537e+05,1.381559,0.254217
min,1.000000e+02,-2.500000e+06,1.0,5.000000e+01,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,...,500.000000,0.000000,1.000000,28.000000,5.000000e+02,0.000000e+00,1.000000e+02,5.000000e+02,0.000000,0.000000
25%,1.000000e+03,-6.000000e+03,1.0,1.095000e+03,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,...,19000.000000,1.000000,8.000000,89.000000,1.782609e+03,8.555886e+02,5.000000e+02,5.000000e+03,0.000000,0.000000
50%,2.000000e+03,-2.000000e+03,1.0,2.228000e+03,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,...,26000.000000,3.000000,16.000000,131.000000,3.193750e+03,1.962430e+03,1.000000e+03,9.500000e+03,0.000000,0.000000
75%,6.000000e+03,-1.000000e+03,1.0,6.300000e+03,1.000000,1.000000,0.000000,1.000000,1.000000,0.0,...,50000.000000,4.000000,24.000000,157.000000,7.225732e+03,3.742979e+03,2.000000e+03,1.350000e+04,0.000000,0.000000
max,2.500000e+06,-1.000000e+02,1.0,2.760000e+06,1.000000,1.000000,1.000000,12.000000,16.000000,0.0,...,1000000.000000,6.000000,31.000000,192.000000,1.250250e+06,1.767413e+06,1.200000e+06,2.500000e+06,12.000000,1.000000
